# Download NFIP Claims File
Author: Mark Bauer

In [1]:
# import libraries
import requests
from datetime import datetime
import duckdb

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p requests,duckdb

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

requests: 2.28.1
duckdb  : 1.0.0



In [3]:
# data retrieved
current_date = datetime.now()
print(f"The data was retrieved on {current_date.strftime('%Y-%m-%d')}.")

The data was retrieved on 2024-10-20.


In [4]:
# URL of the CSV file to download
url = "https://www.fema.gov/about/reports-and-data/openfema/FimaNfipClaims.csv"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    
    # Specify the file path where you want to save the downloaded file
    file_path = "data/FimaNfipClaims.csv"
    
    # Write the content of the response to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")
    
else:
    print("Failed to download file.")

File downloaded successfully.


In [5]:
# confirm file in data/ folder
%ls data/

FimaNfipClaims.csv


In [6]:
# list file sizes
!du -sh * | sort -rh

985M	data
288K	analysis.ipynb
264K	figures
168K	map.ipynb
8.0K	download-data.ipynb
4.0K	README.md
4.0K	LICENSE


In [7]:
# preview size of file
!du -sh data/* | sort -rh

985M	data/FimaNfipClaims.csv


In [8]:
# sanity check on exported file
duckdb.sql("SELECT * FROM read_csv('data/FimaNfipClaims.csv') LIMIT 10;")

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────┬───────────┬──────────────────────┐
│ agricultureStructu…  │       asOfDate       │ basementEnclosureC…  │ … │ latitude │ longitude │          id          │
│        int64         │      timestamp       │        int64         │   │  double  │  double   │       varchar        │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────┼───────────┼──────────────────────┤
│                    0 │ 2020-11-13 14:50:3…  │                    2 │ … │     30.3 │     -87.7 │ e7af3d9f-b605-4653…  │
│                    0 │ 2020-12-11 16:25:4…  │                    0 │ … │     30.3 │     -87.7 │ bbaeaf64-c162-41bf…  │
│                    0 │ 2020-03-27 12:15:4…  │                    0 │ … │     33.5 │    -112.1 │ 256da746-b30b-4129…  │
│                    0 │ 2020-03-27 12:15:4…  │                    0 │ … │     33.5 │    -112.1 │ e3dcbb27-a2d0-4a9e…  │
│                    0 │ 2020-03